1. Initialization

In [ ]:
import ee
import geemap
import logging
import multiprocessing
import os
import requests
import shutil
from retry import retry

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
Map = geemap.Map()

2. Layer Addition (Unique for Each User if Uploaded assest is to be used)

In [ ]:
ds = ee.FeatureCollection("projects/ee-pnolaraj/assets/Ktm_Shp")
Map.addLayer(ds, {}, "Kathmandu")

3. Extraction of CoOrdinates from Matching String to Feature Name or portion of name

In [ ]:
# Provide any string on the name for filtering
def School_CoOrdinates(String):
    filtered_features = ds.filter(ee.Filter.stringContains('name', String))

    # Map over the filtered features to extract their coordinates
    coordinates_list = filtered_features.geometry().coordinates()
    
    #Places and Co ordinates values    
    place_names = filtered_features.aggregate_array('name').getInfo()
    CoOrdinatesList = coordinates_list.getInfo()
   
    
    #Sorting the boundary Cordinates to point
    CoOrdinates_sorted = []
    for i in CoOrdinatesList:
        for j in i:
            for k in j:
                CoOrdinates_sorted.append([k[0], k[1]])
                break
            break
            
            
    return [CoOrdinates_sorted, place_names]

In [ ]:
String = "School"

Result = School_CoOrdinates(String)
CoOrdinate, Place_Name = Result[0], Result[1]

In [ ]:
print(len(Place_Name))
print(CoOrdinate)
print(Place_Name)

4. Adjusting Output Parameters

In [ ]:
params = {
    'count': 100,  # How many image chips to export
    'buffer': 100,  # The buffer distance (m) around each point
    'scale': 100,  # The scale to do stratified sampling
    'seed': 1,  # A randomization seed to use for subsampling.
    'dimensions': '256x256',  # The dimension of each image chip
    'format': "geojson",  # The output image format, can be png, jpg, ZIPPED_GEO_TIFF, GEO_TIFF, NPY
    'prefix': f'{String}_',  # The filename prefix
    'processes': 25,  # How many processes to used for parallel processing
    'out_dir': 'Output',  # The output directory. Default to the current working directly
}

5. Specifying Collection that corresponds to the output image of Interest

In [ ]:
image = ee.FeatureCollection("GOOGLE/Research/open-buildings/v2/polygons")

6. Extraction of Output

In [ ]:
def GEOJSON_from_Point(Point, Name):
    image = ee.FeatureCollection("GOOGLE/Research/open-buildings/v2/polygons")
    point_geom = ee.Geometry.Point(Point[0], Point[1])
    buffer = point_geom.buffer(buffer_radius)
    clipped_collection = image.filterBounds(buffer)
    download_url = clipped_collection.getDownloadURL(filetype='geojson')
    
    r = requests.get(download_url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()
        
        
    ext = params['format']
    out_dir = os.path.abspath(params['out_dir'])    
    basename = str(Name)
    replacements = [",", "/", "\\", "?", "*", ":", "<", ">", "|", '"']
    for character in replacements:
        if character in basename:
                    basename = basename.replace(character, " ")  
    filename = f"{out_dir}/{params['prefix']}{basename}.{ext}"
                    
                    
    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(r.raw, out_file)

    

In [ ]:
Start_From = "Gyan Bhumi Secondary School"
buffer_radius = 30

Total_Numbers = len(Place_Name)
i=1
j = 1
for Point, Name in zip(CoOrdinate, Place_Name):
    if Name == Start_From:
        i = 2
        print(Name)
    if i == 2:
        GEOJSON_from_Point(Point, Name)
        print("Done: ", str(j) + '/' + str(Total_Numbers), Name)
    j += 1


7. Writing CoOrdinates and Feature Name to text file inside Out_dir as specified above in params

In [ ]:
out_dir = os.path.abspath(params['out_dir'])  
TextFile = f"{out_dir}/{params['prefix']}CoOrdinates.txt"
with open(TextFile, 'w') as f:
    for Point, Name in zip(CoOrdinate, Place_Name):
        f.write(str(Point[0]))
        f.write('\t')        
        f.write(str(Point[1]))
        f.write('\t')
        f.write(Name)
        f.write('\n')